In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import h5py
import multiprocessing
from math import pi

from analysis.config import birds, h5_path_dict, pickle_dir
from analysis.ExpData import ExpData
from analysis.utils import popln_overlap, get_fr

In [ ]:
def get_site_distance(a, b):
    dist = min(
        abs(a - b), 16 - abs(a-b)
        ) % 16
    return dist

In [ ]:
with open(pickle_dir / 'population_patterns_checks.p', 'rb') as f: 
    population_patterns = pickle.load(f)

In [ ]:
save_dir = pickle_dir/"reactivation_pairs"

# Runs pairwise overlap comparisons between visit types
For each pairwise comparison, will do this for each pair of sites

### Cache v Cache

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
count = 0
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    for i, c1_hop in enumerate(c_hops):
        c1_site = int(exp_data.event_sites[exp_data.cache_event][i]) - 1
        cache1_pattern_mat = hop_patterns[c1_hop]
        for j, c2_hop in enumerate(c_hops):
            if j <= i: continue
            c2_site = int(exp_data.event_sites[exp_data.cache_event][j]) - 1
            cache2_pattern_mat = hop_patterns[c2_hop]
            overlaps = []
            for cache2_pattern in cache2_pattern_mat.T:
                cache1_cache2_overlap = popln_overlap(
                    cache1_pattern_mat, cache2_pattern
                    )
                max_val = np.nanmax(cache1_cache2_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[c1_site][c2_site].append(max_overlap)
                count += 1
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'cache_cache.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Cache v Retrieval

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
count = 0
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    for i, c_hop in enumerate(c_hops):
        c_site = int(exp_data.event_sites[exp_data.cache_event][i]) - 1
        cache_pattern_mat = hop_patterns[c_hop]
        for j, r_hop in enumerate(r_hops):
            r_site = int(exp_data.event_sites[exp_data.retriev_event][j]) - 1
            retriev_pattern_mat = hop_patterns[r_hop]
            overlaps = []
            for retriev_pattern in retriev_pattern_mat.T:
                cache_retriev_overlap = popln_overlap(
                    cache_pattern_mat, retriev_pattern
                    )
                max_val = np.nanmax(cache_retriev_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[c_site][r_site].append(max_overlap)
                count += 1
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'cache_retriev.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Cache v Full Check

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    if ch_hops.size == 0: continue
    for i, c_hop in enumerate(c_hops):
        c_site = exp_data.event_sites[exp_data.cache_event][i] - 1
        cache_pattern_mat = hop_patterns[c_hop]
        for j, ch_hop in enumerate(ch_hops):
            ch_site = exp_data.event_sites[exp_data.check_event][j] - 1
            if not exp_data.cache_present[ch_hop, ch_site]: continue
            check_pattern_mat = hop_patterns[ch_hop]
            overlaps = []
            for check_pattern in check_pattern_mat.T:
                cache_check_overlap = popln_overlap(
                    cache_pattern_mat, check_pattern
                    )
                max_val = np.nanmax(cache_check_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[c_site][ch_site].append(max_overlap)         
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'cache_checkfull.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Cache v Empty Check

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    if ch_hops.size == 0: continue
    for i, c_hop in enumerate(c_hops):
        c_site = exp_data.event_sites[exp_data.cache_event][i] - 1
        cache_pattern_mat = hop_patterns[c_hop]
        for j, ch_hop in enumerate(ch_hops):
            ch_site = exp_data.event_sites[exp_data.check_event][j] - 1
            if exp_data.cache_present[ch_hop, ch_site]: continue
            check_pattern_mat = hop_patterns[ch_hop]
            overlaps = []
            for check_pattern in check_pattern_mat.T:
                cache_check_overlap = popln_overlap(
                    cache_pattern_mat, check_pattern
                    )
                max_val = np.nanmax(cache_check_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[c_site][ch_site].append(max_overlap)         
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'cache_checkempty.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Cache v Full Nav

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    for i, c_hop in enumerate(c_hops):
        c_site = exp_data.event_sites[exp_data.cache_event][i] - 1
        cache_pattern_mat = hop_patterns[c_hop]
        for j, noncrch_hop in enumerate(noncrch_hops):
            overlaps = []
            noncrch_site = exp_data.hop_end_wedges[noncrch_hop] - 1
            if noncrch_site == 16: continue
            if not exp_data.cache_present[noncrch_hop, noncrch_site]: continue
            navig_pattern_mat = hop_patterns[noncrch_hop]
            overlaps = []
            for navig_pattern in navig_pattern_mat.T:
                cache_nav_overlap = popln_overlap(
                    cache_pattern_mat, navig_pattern
                    )
                max_val = np.nanmax(cache_nav_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[c_site][noncrch_site].append(max_overlap)
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'cache_navfull.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Cache v Empty Nav

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    for i, c_hop in enumerate(c_hops):
        c_site = exp_data.event_sites[exp_data.cache_event][i] - 1
        cache_pattern_mat = hop_patterns[c_hop]
        for j, noncrch_hop in enumerate(noncrch_hops):
            overlaps = []
            noncrch_site = exp_data.hop_end_wedges[noncrch_hop] - 1
            if noncrch_site == 16: continue
            if exp_data.cache_present[noncrch_hop, noncrch_site]: continue
            navig_pattern_mat = hop_patterns[noncrch_hop]
            overlaps = []
            for navig_pattern in navig_pattern_mat.T:
                cache_nav_overlap = popln_overlap(
                    cache_pattern_mat, navig_pattern
                    )
                max_val = np.nanmax(cache_nav_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[c_site][noncrch_site].append(max_overlap)
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'cache_navempty.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Retrieval v Retrieval

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
count = 0
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    for i, r1_hop in enumerate(r_hops):
        r1_site = int(exp_data.event_sites[exp_data.retriev_event][i]) - 1
        retriev1_pattern_mat = hop_patterns[r1_hop]
        for j, r2_hop in enumerate(r_hops):
            if j <= i: continue
            r2_site = int(exp_data.event_sites[exp_data.retriev_event][j]) - 1
            retriev2_pattern_mat = hop_patterns[r2_hop]
            overlaps = []
            for retriev2_pattern in retriev2_pattern_mat.T:
                retriev1_retriev2_overlap = popln_overlap(
                    retriev1_pattern_mat, retriev2_pattern
                    )
                max_val = np.nanmax(retriev1_retriev2_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[r1_site][r2_site].append(max_overlap)
                count += 1
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'retriev_retriev.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Retrieval v Full Check

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    if ch_hops.size == 0: continue
    for i, r_hop in enumerate(r_hops):
        r_site = exp_data.event_sites[exp_data.retriev_event][i] - 1
        retriev_pattern_mat = hop_patterns[r_hop]
        for j, ch_hop in enumerate(ch_hops):
            ch_site = exp_data.event_sites[exp_data.check_event][j] - 1
            if not exp_data.cache_present[ch_hop, ch_site]: continue
            check_pattern_mat = hop_patterns[ch_hop]
            overlaps = []
            for check_pattern in check_pattern_mat.T:
                retriev_check_overlap = popln_overlap(
                    retriev_pattern_mat, check_pattern
                    )
                max_val = np.nanmax(retriev_check_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[r_site][ch_site].append(max_overlap)         
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'retriev_checkfull.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Retrieval v Empty Check

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    if ch_hops.size == 0: continue
    for i, r_hop in enumerate(r_hops):
        r_site = exp_data.event_sites[exp_data.retriev_event][i] - 1
        retriev_pattern_mat = hop_patterns[r_hop]
        for j, ch_hop in enumerate(ch_hops):
            ch_site = exp_data.event_sites[exp_data.check_event][j] - 1
            if exp_data.cache_present[ch_hop, ch_site]: continue
            check_pattern_mat = hop_patterns[ch_hop]
            overlaps = []
            for check_pattern in check_pattern_mat.T:
                retriev_check_overlap = popln_overlap(
                    retriev_pattern_mat, check_pattern
                    )
                max_val = np.nanmax(retriev_check_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[r_site][ch_site].append(max_overlap)    
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'retriev_checkempty.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Retriev v Full Nav

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    for i, r_hop in enumerate(r_hops):
        r_site = exp_data.event_sites[exp_data.retriev_event][i] - 1
        retriev_pattern_mat = hop_patterns[r_hop]
        for j, noncrch_hop in enumerate(noncrch_hops):
            overlaps = []
            noncrch_site = exp_data.hop_end_wedges[noncrch_hop] - 1
            if noncrch_site == 16: continue
            if not exp_data.cache_present[noncrch_hop, noncrch_site]: continue
            navig_pattern_mat = hop_patterns[noncrch_hop]
            overlaps = []
            for navig_pattern in navig_pattern_mat.T:
                retriev_nav_overlap = popln_overlap(
                    retriev_pattern_mat, navig_pattern
                    )
                max_val = np.nanmax(retriev_nav_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[r_site][noncrch_site].append(max_overlap)
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'retriev_navfull.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Retriev v Empty Nav

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    for i, r_hop in enumerate(r_hops):
        r_site = exp_data.event_sites[exp_data.retriev_event][i] - 1
        retriev_pattern_mat = hop_patterns[r_hop]
        for j, noncrch_hop in enumerate(noncrch_hops):
            overlaps = []
            noncrch_site = exp_data.hop_end_wedges[noncrch_hop] - 1
            if noncrch_site == 16: continue
            if exp_data.cache_present[noncrch_hop, noncrch_site]: continue
            navig_pattern_mat = hop_patterns[noncrch_hop]
            overlaps = []
            for navig_pattern in navig_pattern_mat.T:
                retriev_nav_overlap = popln_overlap(
                    retriev_pattern_mat, navig_pattern
                    )
                max_val = np.nanmax(retriev_nav_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[r_site][noncrch_site].append(max_overlap)
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'retriev_navempty.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Empty Check v Empty Check

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    if ch_hops.size == 0: continue
    for i, ch1_hop in enumerate(ch_hops):
        ch1_site = exp_data.event_sites[exp_data.check_event][i] - 1
        if exp_data.cache_present[ch1_hop, ch1_site]: continue
        check1_pattern_mat = hop_patterns[ch1_hop]
        for j, ch2_hop in enumerate(ch_hops):
            if j <= i: continue
            ch2_site = exp_data.event_sites[exp_data.check_event][j] - 1
            if exp_data.cache_present[ch2_hop, ch2_site]: continue
            check2_pattern_mat = hop_patterns[ch2_hop]
            overlaps = []
            for check2_pattern in check2_pattern_mat.T:
                check1_check2_overlap = popln_overlap(
                    check1_pattern_mat, check2_pattern
                    )
                max_val = np.nanmax(check1_check2_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[ch1_site][ch2_site].append(max_overlap)    
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'checkempty_checkempty.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Empty Check v Full Check

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    if ch_hops.size == 0: continue
    for i, ch1_hop in enumerate(ch_hops):
        ch1_site = exp_data.event_sites[exp_data.check_event][i] - 1
        if exp_data.cache_present[ch1_hop, ch1_site]: continue
        check1_pattern_mat = hop_patterns[ch1_hop]
        for j, ch2_hop in enumerate(ch_hops):
            if j <= i: continue
            ch2_site = exp_data.event_sites[exp_data.check_event][j] - 1
            if not exp_data.cache_present[ch2_hop, ch2_site]: continue
            check2_pattern_mat = hop_patterns[ch2_hop]
            overlaps = []
            for check2_pattern in check2_pattern_mat.T:
                check1_check2_overlap = popln_overlap(
                    check1_pattern_mat, check2_pattern
                    )
                max_val = np.nanmax(check1_check2_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[ch1_site][ch2_site].append(max_overlap)    
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'checkempty_checkfull.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Empty Check v Empty Nav

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    if ch_hops.size == 0: continue
    for i, ch_hop in enumerate(ch_hops):
        ch_site = exp_data.event_sites[exp_data.check_event][i] - 1
        if exp_data.cache_present[ch_hop, ch_site]: continue
        check_pattern_mat = hop_patterns[ch_hop]
        for j, noncrch_hop in enumerate(noncrch_hops):
            overlaps = []
            noncrch_site = exp_data.hop_end_wedges[noncrch_hop] - 1
            if noncrch_site == 16: continue
            if exp_data.cache_present[noncrch_hop, noncrch_site]: continue
            navig_pattern_mat = hop_patterns[noncrch_hop]
            overlaps = []
            for navig_pattern in navig_pattern_mat.T:
                check_nav_overlap = popln_overlap(
                    check_pattern_mat, navig_pattern
                    )
                max_val = np.nanmax(check_nav_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[ch_site][noncrch_site].append(max_overlap)
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'checkempty_navempty.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Empty Check v Full Nav

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    if ch_hops.size == 0: continue
    for i, ch_hop in enumerate(ch_hops):
        ch_site = exp_data.event_sites[exp_data.check_event][i] - 1
        if exp_data.cache_present[ch_hop, ch_site]: continue
        check_pattern_mat = hop_patterns[ch_hop]
        for j, noncrch_hop in enumerate(noncrch_hops):
            overlaps = []
            noncrch_site = exp_data.hop_end_wedges[noncrch_hop] - 1
            if noncrch_site == 16: continue
            if not exp_data.cache_present[noncrch_hop, noncrch_site]: continue
            navig_pattern_mat = hop_patterns[noncrch_hop]
            overlaps = []
            for navig_pattern in navig_pattern_mat.T:
                check_nav_overlap = popln_overlap(
                    check_pattern_mat, navig_pattern
                    )
                max_val = np.nanmax(check_nav_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[ch_site][noncrch_site].append(max_overlap)
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'checkempty_navfull.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Full Check v Full Check

In [ ]:
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for fil_string in population_patterns.keys():
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    if ch_hops.size == 0: continue
    for i, ch1_hop in enumerate(ch_hops):
        ch1_site = exp_data.event_sites[exp_data.check_event][i] - 1
        if not exp_data.cache_present[ch1_hop, ch1_site]: continue
        check1_pattern_mat = hop_patterns[ch1_hop]
        for j, ch2_hop in enumerate(ch_hops):
            if j <= i: continue
            ch2_site = exp_data.event_sites[exp_data.check_event][j] - 1
            if not exp_data.cache_present[ch2_hop, ch2_site]: continue
            check2_pattern_mat = hop_patterns[ch2_hop]
            overlaps = []
            for check2_pattern in check2_pattern_mat.T:
                check1_check2_overlap = popln_overlap(
                    check1_pattern_mat, check2_pattern
                    )
                max_val = np.nanmax(check1_check2_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[ch1_site][ch2_site].append(max_overlap)    
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'checkfull_checkfull.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Full Check v Empty Nav

In [ ]:
def get_distance_mat(fil_string):
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
    if ch_hops.size == 0: return np.array(distance_matrix)
    for i, ch_hop in enumerate(ch_hops):
        ch_site = exp_data.event_sites[exp_data.check_event][i] - 1
        if not exp_data.cache_present[ch_hop, ch_site]: continue
        check_pattern_mat = hop_patterns[ch_hop]
        for j, noncrch_hop in enumerate(noncrch_hops):
            overlaps = []
            noncrch_site = exp_data.hop_end_wedges[noncrch_hop] - 1
            if noncrch_site == 16: continue
            if exp_data.cache_present[noncrch_hop, noncrch_site]: continue
            navig_pattern_mat = hop_patterns[noncrch_hop]
            overlaps = []
            for navig_pattern in navig_pattern_mat.T:
                check_nav_overlap = popln_overlap(
                    check_pattern_mat, navig_pattern
                    )
                max_val = np.nanmax(check_nav_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[ch_site][noncrch_site].append(max_overlap)
    return np.array(distance_matrix)

In [ ]:
# Run in parallel
PROCESSES = 5
distance_matrices = []
with multiprocessing.Pool(PROCESSES) as pool:
    params = [p for p in population_patterns.keys()]
    for res in pool.map(get_distance_mat, params):
        distance_matrices.append(res)

In [ ]:
# Format into the same structure as before
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for mat in distance_matrices:
    for i in range(16):
        for j in range(16):
            distance_matrix[i][j].extend(mat[i][j])
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'checkfull_navempty.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Full Check v Full Nav

In [ ]:
def get_distance_mat(fil_string):
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
    if ch_hops.size == 0: np.array(distance_matrix)
    for i, ch_hop in enumerate(ch_hops):
        ch_site = exp_data.event_sites[exp_data.check_event][i] - 1
        if not exp_data.cache_present[ch_hop, ch_site]: continue
        check_pattern_mat = hop_patterns[ch_hop]
        for j, noncrch_hop in enumerate(noncrch_hops):
            overlaps = []
            noncrch_site = exp_data.hop_end_wedges[noncrch_hop] - 1
            if noncrch_site == 16: continue
            if not exp_data.cache_present[noncrch_hop, noncrch_site]: continue
            navig_pattern_mat = hop_patterns[noncrch_hop]
            overlaps = []
            for navig_pattern in navig_pattern_mat.T:
                check_nav_overlap = popln_overlap(
                    check_pattern_mat, navig_pattern
                    )
                max_val = np.nanmax(check_nav_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[ch_site][noncrch_site].append(max_overlap)
    return np.array(distance_matrix)

In [ ]:
# Run in parallel
PROCESSES = 5
distance_matrices = []
with multiprocessing.Pool(PROCESSES) as pool:
    params = [p for p in population_patterns.keys()]
    for res in pool.map(get_distance_mat, params):
        distance_matrices.append(res)

In [ ]:
# Format into the same structure as before
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for mat in distance_matrices:
    for i in range(16):
        for j in range(16):
            distance_matrix[i][j].extend(mat[i][j])
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'checkfull_navfull.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Empty Nav v Empty Nav

In [ ]:
def get_distance_mat(fil_string):
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
    for i, noncrch1_hop in enumerate(noncrch_hops):
        noncrch1_site = exp_data.hop_end_wedges[noncrch1_hop] - 1
        if noncrch1_site == 16: continue
        if exp_data.cache_present[noncrch1_hop, noncrch1_site]: continue
        noncrch1_pattern_mat = hop_patterns[noncrch1_hop]
        for j, noncrch2_hop in enumerate(noncrch_hops):
            if j <= i: continue
            overlaps = []
            noncrch2_site = exp_data.hop_end_wedges[noncrch2_hop] - 1
            if noncrch2_site == 16: continue
            if exp_data.cache_present[noncrch2_hop, noncrch2_site]: continue
            navig2_pattern_mat = hop_patterns[noncrch2_hop]
            overlaps = []
            for navig2_pattern in navig2_pattern_mat.T:
                nav1_nav2_overlap = popln_overlap(
                    noncrch1_pattern_mat, navig2_pattern
                    )
                max_val = np.nanmax(nav1_nav2_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[noncrch1_site][noncrch2_site].append(max_overlap)
    return np.array(distance_matrix)

In [ ]:
# Run in parallel
PROCESSES = 5
distance_matrices = []
with multiprocessing.Pool(PROCESSES) as pool:
    params = [p for p in population_patterns.keys()]
    for res in pool.map(get_distance_mat, params):
        distance_matrices.append(res)

In [ ]:
# Format into the same structure as before
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for mat in distance_matrices:
    for i in range(16):
        for j in range(16):
            distance_matrix[i][j].extend(mat[i][j])
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'navempty_navempty.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Empty Nav v Full Nav

In [ ]:
def get_distance_matrix(fil_string):
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
    for i, noncrch1_hop in enumerate(noncrch_hops):
        noncrch1_site = exp_data.hop_end_wedges[noncrch1_hop] - 1
        if noncrch1_site == 16: continue
        if exp_data.cache_present[noncrch1_hop, noncrch1_site]: continue
        noncrch1_pattern_mat = hop_patterns[noncrch1_hop]
        for j, noncrch2_hop in enumerate(noncrch_hops):
            if j <= i: continue
            overlaps = []
            noncrch2_site = exp_data.hop_end_wedges[noncrch2_hop] - 1
            if noncrch2_site == 16: continue
            if not exp_data.cache_present[noncrch2_hop, noncrch2_site]: continue
            navig2_pattern_mat = hop_patterns[noncrch2_hop]
            overlaps = []
            for navig2_pattern in navig2_pattern_mat.T:
                nav1_nav2_overlap = popln_overlap(
                    noncrch1_pattern_mat, navig2_pattern
                    )
                max_val = np.nanmax(nav1_nav2_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[noncrch1_site][noncrch2_site].append(max_overlap)
    return np.array(distance_matrix)

In [ ]:
# Run in parallel
PROCESSES = 5
distance_matrices = []
with multiprocessing.Pool(PROCESSES) as pool:
    params = [p for p in population_patterns.keys()]
    for res in pool.map(get_distance_mat, params):
        distance_matrices.append(res)

In [ ]:
# Format into the same structure as before
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for mat in distance_matrices:
    for i in range(16):
        for j in range(16):
            distance_matrix[i][j].extend(mat[i][j])
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'navempty_navfull.p', 'wb') as f:
    pickle.dump(distance_matrix, f)

### Full Nav v Full Nav

In [ ]:
def get_distance_matrix(fil_string):
    f = h5py.File(fil_string, 'r')
    exp_data = ExpData(f)
    c_hops = population_patterns[fil_string]['c_hops']
    r_hops = population_patterns[fil_string]['r_hops']
    ch_hops = population_patterns[fil_string]['ch_hops']
    noncrch_hops = population_patterns[fil_string]['noncrch_hops']
    hop_patterns = population_patterns[fil_string]['hop_patterns']
    distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
    for i, noncrch1_hop in enumerate(noncrch_hops):
        noncrch1_site = exp_data.hop_end_wedges[noncrch1_hop] - 1
        if noncrch1_site == 16: continue
        if not exp_data.cache_present[noncrch1_hop, noncrch1_site]: continue
        noncrch1_pattern_mat = hop_patterns[noncrch1_hop]
        for j, noncrch2_hop in enumerate(noncrch_hops):
            if j <= i: continue
            overlaps = []
            noncrch2_site = exp_data.hop_end_wedges[noncrch2_hop] - 1
            if noncrch2_site == 16: continue
            if not exp_data.cache_present[noncrch2_hop, noncrch2_site]: continue
            navig2_pattern_mat = hop_patterns[noncrch2_hop]
            overlaps = []
            for navig2_pattern in navig2_pattern_mat.T:
                nav1_nav2_overlap = popln_overlap(
                    noncrch1_pattern_mat, navig2_pattern
                    )
                max_val = np.nanmax(nav1_nav2_overlap)
                if not np.isnan(max_val):
                    overlaps.append(max_val)
            if len(overlaps) > 0:
                max_overlap = np.nanmax(overlaps)
                distance_matrix[noncrch1_site][noncrch2_site].append(max_overlap)
    return np.array(distance_matrix)

In [ ]:
# Run in parallel
PROCESSES = 5
distance_matrices = []
with multiprocessing.Pool(PROCESSES) as pool:
    params = [p for p in population_patterns.keys()]
    for res in pool.map(get_distance_mat, params):
        distance_matrices.append(res)

In [ ]:
# Format into the same structure as before
distance_matrix = [[[] for _ in range(16)] for _ in range(16)]
for mat in distance_matrices:
    for i in range(16):
        for j in range(16):
            distance_matrix[i][j].extend(mat[i][j])
distance_matrix = np.array(distance_matrix)

In [ ]:
with open(save_dir/'navfull_navfull.p', 'wb') as f:
    pickle.dump(distance_matrix, f)